# EDA simple project

## Preparations

In [554]:
# import all libraries/packages
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# for geo plotting
import plotly.express as px
import altair as alt
from vega_datasets import data


# from matplotlib.ticker import PercentFormatter
# plt.rcParams.update({ "figure.figsize" : (8, 5),"axes.facecolor" : "white", "axes.edgecolor":  "black"})
# plt.rcParams["figure.facecolor"]= "w"
# pd.plotting.register_matplotlib_converters()
# pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [503]:
# import dataset
df = pd.read_csv('data/King_County_House_prices_dataset.csv')

## Understand the data

In [555]:
# print out the shape and the head of the dataset
print(df.shape)
df.head(2)

(21597, 21)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639


In [506]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

Quickly check each column, following clean up processes need to be done. ()

  Data Types:
| Column name   | Clean up |  Description |
| ---           | ---      | ----------- |
| id            | Drop            | unique identified for a house|
| date          | change to data type| house was sold |
| price         | drop the digits behind, set to int| is prediction target |
| bedrooms      | OK |  |
| bathrooms     | OK|  |
| sqft_living   |  |  |
| sqft_lot      |   | |
| floors        | round to 1 digit | values are array([1. , 2. , 1.5, 3. , 2.5, 3.5]) |
| waterfront    | Drop | House which has a view to a waterfront |
| view          |  | |
| conditions    |  | How good the condition is ( Overall ) |
| grade         |  | overall grade given to the housing unit, based on King County grading system |
| sqft_above    |  | square footage of house apart from basement |
| sqft_basement | str->int type, some values need to be replaced | square footage of the basement |
| yr_built      | change to year |  |
| yr_renovated  | clean up |  |
| zipcode       | OK |  |
| lat           | OK |  |
| long          | OK | |
| sqft_living15 | interesting | The square footage of interior housing living space for the nearest 15 neighbors |
| sqft_lot15    | interesting | The square footage of the land lots of the nearest 15 neighbors |

- Object: `date` needs to be transferred to Date, `sqft_basement` -> int64
- float64: 
- Date type: `date`, `yr_built`,`yr_renovated`
- There are NO duplicated rows found

# Data cleansing

In [568]:
dup_id = df[df['id'].duplicated()].id
df_dup = df[df.id.isin(dup_id)]
idx1 = df[df['id'].duplicated(keep=False)].index
len(idx1)

353

In [577]:
# check duplicated data
idx2 = df[df[['lat','long','zipcode']].duplicated(keep=False)].index

idx_dup_loc = [i for i in idx2 if i not in idx1]

df.iloc[idx_dup_loc].sort_values(by=['lat']).head(20)
# df_dup_loc = df[(df.lat.isin(dup_loc)) & (df.long.isin(df[df[['lat','long']].duplicated()].long))]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
20909,8141310040,6/27/2014,246950.0,3,3.00,1670,4440,1.0,0.0,0.0,...,7,1670,0.0,2014,0.0,98022,47.1948,-121.975,1670,4622
20670,8141310030,7/30/2014,256703.0,3,2.00,1670,4441,1.0,0.0,0.0,...,7,1670,0.0,2014,0.0,98022,47.1948,-121.975,1670,4622
16521,5595900345,1/13/2015,460000.0,4,2.75,3460,13168,2.0,0.0,0.0,...,8,3460,0.0,1932,1986.0,98022,47.2046,-121.996,1500,7670
5142,5595900280,3/18/2015,235000.0,3,1.00,1050,7670,1.5,NaN,0.0,...,7,1050,0.0,1955,0.0,98022,47.2046,-121.996,1220,7670
8422,9432900180,7/14/2014,307999.0,4,2.75,2420,8438,2.0,0.0,0.0,...,8,2420,0.0,1996,0.0,98022,47.2091,-122.009,2420,8580
7163,9432900380,10/23/2014,280017.0,3,2.50,1850,8770,2.0,0.0,0.0,...,8,1850,0.0,1996,0.0,98022,47.2091,-122.009,2350,8606
8122,2420069278,3/19/2015,287000.0,3,2.50,1820,8722,1.5,0.0,0.0,...,7,1820,0.0,1926,2008.0,98022,47.2137,-121.989,1480,12285
14772,2420069604,3/30/2015,255000.0,3,2.50,1720,6200,2.0,0.0,0.0,...,7,1720,0.0,2014,0.0,98022,47.2137,-121.989,1710,9520
13276,3750605674,9/17/2014,270000.0,3,2.50,1808,19200,1.0,0.0,0.0,...,8,1808,0.0,2005,0.0,98001,47.2598,-122.281,1450,14400
7897,3750605620,3/24/2015,225000.0,3,1.75,1580,14400,1.0,NaN,0.0,...,7,1580,0.0,1981,0.0,98001,47.2598,-122.281,1480,9600


In [531]:
df[df[['lat','long','zipcode']].duplicated()]

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
94,6021501535,12/23/2014,700000.0,3,1.50,1580,5000,1.0,0.0,0.0,...,8,1290,290.0,1939,0.0,98117,47.6870,-122.386,1570,4500
314,4139480200,12/9/2014,1400000.0,4,3.25,4290,12103,1.0,0.0,3.0,...,11,2690,1600.0,1997,0.0,98006,47.5503,-122.102,3860,11244
325,7520000520,3/11/2015,240500.0,2,1.00,1240,12092,1.0,0.0,0.0,...,6,960,280.0,1922,1984.0,98146,47.4957,-122.352,1820,7460
346,3969300030,12/29/2014,239900.0,4,1.00,1000,7134,1.0,0.0,0.0,...,6,1000,0.0,1943,NaN,98178,47.4897,-122.240,1020,7138
372,2231500030,3/24/2015,530000.0,4,2.25,2180,10754,1.0,0.0,0.0,...,7,1100,1080.0,1954,0.0,98133,47.7711,-122.341,1810,6929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21579,1972201967,10/31/2014,520000.0,2,2.25,1530,981,3.0,0.0,0.0,...,8,1480,50.0,2006,NaN,98103,47.6533,-122.346,1530,1282
21582,8956200760,10/13/2014,541800.0,4,2.50,3118,7866,2.0,NaN,2.0,...,9,3118,0.0,2014,0.0,98001,47.2931,-122.264,2673,6500
21591,2997800021,2/19/2015,475000.0,3,2.50,1310,1294,2.0,0.0,0.0,...,8,1180,130.0,2008,0.0,98116,47.5773,-122.409,1330,1265
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,...,8,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509


## Drop irrelevant columns

In [ ]:
'''Drop columns not needed
view: only about 10% houses are viewed, so it is not really representative regarding popularity 
'''
df = df.drop(['id' ,'waterfront', 'view'], axis=1)

## Data type and transforming

**Date entry**: `date` and `yr_built` `yr_renovated`

In [ ]:
df.head()

In [ ]:
# find out NaN value in renovated year
pd.isna(df.yr_renovated).value_counts()

In [ ]:
# find out how many houses are not renovated
df.query('yr_renovated==0').date.count()

In [ ]:
# Change date to date type
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')
df['yr_built'] = pd.to_datetime(df['yr_built'], format='%Y').dt.year

For renovation year, because there are NaN and 0 data, we use `errors='ignore'` to solve the issue

In [ ]:
# for year in df.yr_renovated, be
df['yr_renovated'] = pd.to_datetime(df['yr_renovated'], format='%Y.0', errors='ignore')

Casting to int

In [ ]:
# casting price  to int so long there is no missing value
df.price = df.price.astype(int)


Col `floors`

In [ ]:
df.floors.unique()
df.floors.dtype

In [ ]:
df.floors = df.floors.round(2)
df.head()

Change `grade` and `condition` as categorical type

In [ ]:
df.condition = df.condition.astype('category')
df.grade = df.grade.astype('category')

`sqft_basement` is `str`type, find out special characters contained inside and transform it to int

In [ ]:
# df.sqft_basement.astype(float, errors="ignore")
df.sqft_basement = df.sqft_basement.str.replace('?', '0')

In [ ]:
# change type to float and then to int
df.sqft_basement = df.sqft_basement.astype(float)
df.sqft_basement = df.sqft_basement.astype(int)
df.sqft_basement.dtype

### Handling Missing Data

In [ ]:
# check results
df.info()

# First glance at data

In [ ]:
# set sns theme
sns.set_theme(style="ticks")

Housing price distribution

In [ ]:
print(df[['price','sqft_living','sqft_lot','sqft_living15','sqft_lot15']].describe(percentiles=[.1, .5, .75]))
a = df['price'].median()
print(f'median value is: {a}')

In [ ]:
df_price50 = df[df.price<df.price.describe()['50%']]
zip_chp = df_price50.zipcode.nunique()

zip_chp

In [ ]:
sns.set_theme(style="ticks")
# f, ax = plt.subplots(figsize=(7, 6))

sns.displot(data=df, x="price", kde=True, hue='condition')

plt.title('Housing Price Distribution')
plt.xlabel('Price in dollor')
plt.ylabel('Number')
# plt.xlim((0,2555555))
# plt.ylim((0,50))

- Seems that condition 3 has higher mean price

Exam numerical data distribution
- Set a list of all types of our data in the dataset and only take the numerical one
- Plot the distribution

In [ ]:
list(set(df.dtypes.tolist()))

df_num = df.select_dtypes(include=['float64','int64'])
df_num.head()

df_num.hist(figsize=(16, 20), bins=40, xlabelsize=8, ylabelsize=8)

In [ ]:
# find if the grade is a indicator for housing price
sns.boxenplot(df, x='grade', y='price')

plt.title('Housing Price vs Grade')
plt.xlabel('Grade')
plt.ylabel('House price in dollor')

Bedroom number, living area size, sqft_living15 seems to have correlation with `price`

In [ ]:
# calculate the correlation with price
df_num_corr = df_num.corr()['price'][1:]
df_num_corr

In [ ]:
df['sold_year'] = df['date'].dt.year
df['sold_month'] = df['date'].dt.month.astype('category')

df.sold_month.head()

- Strongest is with `sqft_living`
- Meaning housing price does not only depend on one factor.
- Next step is to check the location of the houses

## Data analysis Based on Hyposisi

Define the poor neighbor: `n` lowest mean housing price grouped by `zipcode`

In [ ]:
# defind lowest n mean value of houses
n = 10

# Find out the lowest housing price area according to the zipcode
zip_chp = df.groupby('zipcode').price.mean().nsmallest(n).index
zip_chp 
# filter out the dataset of the area
df_chp_area = df[df.zipcode.isin(zip_chp)]
print(df_chp_area.shape)
df_chp_area.describe()

### Geo plotting

# The Task
- **AT LEAST 3 insights** regarding the overall data. One should be geographical.
- **AT LEAST 3 recommendations** for your stakeholder.

Dataset: 
- choose the zip code area with the lower housing price (mean)

In [ ]:
# focus on the lower 50% percentile
df_price50 = df[df.price<df.price.describe()['50%']]
num_zip = df_price50.zipcode.nunique()
print(f'there are {num_zip} zip codes in the lower 50 percentile of housing price')

# defind lowest n mean value of houses
n = 30

# Find out the lowest housing price area according to the zipcode
zip_chp = df_price50.groupby('zipcode').price.mean().nsmallest(n).index

# filter out the dataset of the area
df_chp_area = df_price50[df_price50.zipcode.isin(zip_chp)]
print(df_chp_area.shape)
df_chp_area.describe()

In [ ]:
### Plotly to plot geo plot
import plotly.graph_objects as go

fig = px.scatter_mapbox(
                        df_chp_area, lat="lat", lon="long",
                        hover_name="price", hover_data=["sqft_living", "condition"],
                        color=df_chp_area.price,
                        color_discrete_sequence=px.colors.qualitative.G10,
                        zoom=8.5, width=650, height=400, 
                        opacity=0.6
                        )

# fig.update_traces(name={'House Distribution', 'test'})
fig.update_layout(mapbox_style="stamen-terrain")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Insights

### Insights 1
There are certain month the real estate market is especially active, summer?

### Insights 2
when to buy the house

### Insights 3
where to buy

### Insights 4
What to look for?

## Recommendations